## Basic Generator Usage

In this tutorial, we will show four ways to use generator:
1. Use directly by setting up a ``ModelClient``.
2. Customize prompt template using ``jinjia2``.
3. Try different models.
4. Use ``acall`` to do mutiple asynchronous calls for speed up.

In default, the generator uses a default prompt template. It has these varaibles:

```
LIGHTRAG_DEFAULT_PROMPT_ARGS = [
    "task_desc_str",
    "output_format_str",
    "tools_str",
    "examples_str",
    "chat_history_str",
    "context_str",
    "steps_str",
    "input_str",
    "output_str",
]
```

In [1]:
# first, let's set up the library log just in case, in default at INFO level
from adalflow.utils.logger import get_logger

get_logger()

<RootLogger root (INFO)>

In [2]:
from adalflow.core import Generator
from adalflow.components.model_client import OpenAIClient
from adalflow.utils import setup_env  # ensure you have .env with OPENAI_API_KEY

setup_env(".env")
query = "What is the capital of France?"
model_kwargs = {"model": "gpt-3.5-turbo"}
generator = Generator(model_client=OpenAIClient(), model_kwargs=model_kwargs)
prompt_kwargs = {
    "input_str": query,
}
# run the generator
output = generator(prompt_kwargs=prompt_kwargs)
print(output)

2024-09-11 06:28:35 - prompt_builder - INFO - [prompt_builder.py:65:__init__] - Prompt has variables: ['steps_str', 'context_str', 'output_format_str', 'input_format_str', 'examples_str', 'chat_history_str', 'task_desc_str', 'tools_str', 'input_str']
2024-09-11 06:28:35 - generator - INFO - [generator.py:141:__init__] - Generator Generator initialized.
2024-09-11 06:28:35 - openai_client - INFO - [openai_client.py:279:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<START_OF_SYSTEM_PROMPT>\nYou are a helpful assistant.\n<END_OF_SYSTEM_PROMPT>\n<START_OF_USER_PROMPT>\nWhat is the capital of France?\n<END_OF_USER_PROMPT>'}]}
2024-09-11 06:28:36 - _client - INFO - [_client.py:1038:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-11 06:28:36 - generator - INFO - [generator.py:773:call] - output: GeneratorOutput(id=None, data='The capital of France is Paris.', error=None, usage=Completi

The logging clearly shows us what we sent to OpenAI.

In [3]:
# lets see the prompt, it is quite minimal
generator.print_prompt(**prompt_kwargs)

Prompt:
______________________
<START_OF_SYSTEM_PROMPT>
You are a helpful assistant.
<END_OF_SYSTEM_PROMPT>
<START_OF_USER_PROMPT>
What is the capital of France?
<END_OF_USER_PROMPT>



'<START_OF_SYSTEM_PROMPT>\nYou are a helpful assistant.\n<END_OF_SYSTEM_PROMPT>\n<START_OF_USER_PROMPT>\nWhat is the capital of France?\n<END_OF_USER_PROMPT>\n'

Writing your template is easy. Let us use our own template. Let's say, we want to set up our AI with a sense of humor.

In [4]:
template = """<SYS> Your are an assistant with a great sense of humor.</SYS> User: {{input_str}}. You:"""

generator2 = Generator(
    model_client=OpenAIClient(), model_kwargs=model_kwargs, template=template
)
response = generator2(prompt_kwargs=prompt_kwargs)
print(response)

2024-09-11 06:29:25 - prompt_builder - INFO - [prompt_builder.py:65:__init__] - Prompt has variables: ['input_str']
2024-09-11 06:29:25 - generator - INFO - [generator.py:141:__init__] - Generator Generator initialized.
2024-09-11 06:29:25 - openai_client - INFO - [openai_client.py:279:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> Your are an assistant with a great sense of humor.</SYS> User: What is the capital of France?. You:'}]}
2024-09-11 06:29:26 - _client - INFO - [_client.py:1038:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-11 06:29:26 - generator - INFO - [generator.py:773:call] - output: GeneratorOutput(id=None, data='I\'m not sure, but I\'ve heard it\'s pronounced "Paris". ', error=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=34, total_tokens=50), raw_response='I\'m not sure, but I\'ve heard it\'s pronounced "Paris". ', metadata=None)
Gener

In [5]:
# Let us use llama3 from groq
from adalflow.components.model_client import GroqAPIClient

groq_model_kwargs = {"model": "llama3-8b-8192"}
generator3 = Generator(
    model_client=GroqAPIClient(), model_kwargs=groq_model_kwargs, template=template
)

response = generator3(prompt_kwargs=prompt_kwargs)
print(response)

2024-09-11 06:29:54 - prompt_builder - INFO - [prompt_builder.py:65:__init__] - Prompt has variables: ['input_str']
2024-09-11 06:29:54 - generator - INFO - [generator.py:141:__init__] - Generator Generator initialized.
2024-09-11 06:29:54 - _client - INFO - [_client.py:1038:_send_single_request] - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-11 06:29:54 - generator - INFO - [generator.py:773:call] - output: GeneratorOutput(id=None, data='Bonjour! But let me guess, you\'re not asking me because you just want to chat (although, let\'s be real, I\'m fabulous at conversation). No, I\'m guessing you\'re asking because you need to polish off your geography skills and you\'ve got a Jeopardy! audition coming up, oui?\n\nSo, the capital of France (drumroll, please)... is PARIS! Voilà! You\'re welcome. Now, if you\'ll excuse me, I have to go practice my "Ooh la la"s in the mirror.', error=None, usage=CompletionUsage(completion_tokens=114, prompt_t

In [6]:
# Lets do 10 async calls at once, lets use GroqAPIClient
import nest_asyncio  # import asyncio, use nest_asyncio.apply() if you are in jupyter notebook
import asyncio

nest_asyncio.apply()

import time
from typing import List


async def make_async_calls(queries: List[str]):
    calls = [generator3.acall(prompt_kwargs={"input_str": query}) for query in queries]
    responses = await asyncio.gather(*calls)
    return responses


queries = [query] * 10
start = time.time()
responses = asyncio.run(make_async_calls(queries))
print(f"Time taken for 10 async calls: {time.time() - start}")
print(responses)

2024-09-11 06:30:13 - generator - INFO - [generator.py:789:acall] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-09-11 06:30:13 - generator - INFO - [generator.py:790:acall] - model_kwargs: {}
2024-09-11 06:30:13 - generator - INFO - [generator.py:789:acall] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-09-11 06:30:13 - generator - INFO - [generator.py:790:acall] - model_kwargs: {}
2024-09-11 06:30:13 - generator - INFO - [generator.py:789:acall] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-09-11 06:30:13 - generator - INFO - [generator.py:790:acall] - model_kwargs: {}
2024-09-11 06:30:13 - generator - INFO - [generator.py:789:acall] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-09-11 06:30:13 - generator - INFO - [generator.py:790:acall] - model_kwargs: {}
2024-09-11 06:30:13 - generator - INFO - [generator.py:789:acall] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-